# Projet 9 : Entreprise "La poule qui chante"

# Sommaire
* [Gestion des données](#chapter1)

    * [Import des données](#section_1_1)
    
    * [Vérification des données](#section_1_2)
        * [Description des données](#section_1_2_1)
        * [Organisation des données](#section_1_2_2)
        * [Vérification des doublons](#section_1_2_3)
        * [Vérification des Nan](#section_1_2_4)
        
    * [Organisation des données](#section_1_3)
        * [Pivot](#section_1_3_1)
        * [Fusion des données](#section_1_3_2)
        * [Gestion des outliers](#section_1_3_3)
        
* [Analyse composantes - clusters](#chapter2)

    * [Analyse des composantes](#section_2_1)
        * [Analyse des Composantes Principales (ACP)](#section_2_1_1)
        * [Biplot et charges factorielles](#section_2_1_2)
        * [Éboulis des valeurs propres et coude](#section_2_1_3)
        
    * [Analyse des clusters](#section_2_2)
        * [Analyse via des modèles arbitraires](#section_2_2_1)
        * [Dendrogramme, ou Classification Ascendante Hiérarchique](#section_2_2_2)
        * [Algorithme Kmeans](#section_2_2_3)
        * [Analyse des résultats](#section_2_2_4)
        * [Évaluation du cluster conservé](#section_2_2_5)

# Gestion des données <a class="anchor" id="chapter1"></a>

## Import des données <a class="anchor" id="section_1_1"></a>

In [1]:
# import des modules
from scipy import stats
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import MiniBatchKMeans
from scipy.cluster.hierarchy import cut_tree

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import pycountry

In [2]:
# import des données
dispo_alim = pd.read_csv (r'C:\Users\Avenmythril\Desktop\Formations\DAn\Projet 9\DAN-P9-data\DisponibiliteAlimentaire_2017.csv')
population = pd.read_csv (r'C:\Users\Avenmythril\Desktop\Formations\DAn\Projet 9\DAN-P9-data\Population_2000_2018.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Avenmythril\\Desktop\\Formations\\DAn\\Projet 9\\DAN-P9-data\\DisponibiliteAlimentaire_2017.csv'

## Vérification des données <a class="anchor" id="section_1_2"></a>

### Description des données <a class="anchor" id="section_1_2_1"></a>

In [ ]:
# on effectue un describe sur chaque fichier
dispo_alim.describe()

Cette étape nous permet de repérer différenes colonnes ayant des outliers, qu'il faudra donc gérer plus tard.

In [ ]:
# on effectue aussi un head sur chaque fichier avant de trier
dispo_alim.head()

### Organisation des données <a class="anchor" id="section_1_2_2"></a>

Ici, nous allons préparer les données pertinentes avant la fusion, et notamment supprimer les années autres que 2017 car c'est la seule année présente dans le df dispo_alim. Nous allons aussi éliminer les colonnes inutiles dans notre recherche.

In [ ]:
# on supprime les colonnes inutiles
columns_to_drop = ['Code zone', 'Code Élément', 'Élément', 'Unité', 'Code année', 'Code Domaine', 'Domaine', 'Symbole', 'Description du Symbole', 'Note', 'Code Produit', 'Produit']
population = population.drop(columns=columns_to_drop)
population.head()

In [ ]:
# on filtre les lignes où l'année est égale à 2017, car c'est la seule année présente dans l'autre df
population = population[population['Année'] == 2017]
population.head()

In [ ]:
# suppression des colonnes inutiles avant fusion
columns_to_drop = ['Code Domaine', 'Domaine', 'Code zone', 'Unité', 'Code Élément', 'Code Produit', 'Code année', 'Symbole', 'Description du Symbole']
dispo_alim = dispo_alim.drop(columns=columns_to_drop)
dispo_alim.head()

Dernier point : nous allons supprimer les "Produit" autres que la viande de volaille et les oeufs, qui sont les seuls produits concernant notre entreprise.

In [ ]:
# création du df contenant uniquement les produits qui nous intéressent
produits_volaille = dispo_alim[dispo_alim['Produit'].isin(['Viande de Volailles', 'Oeufs'])]

# tri du df pour un tri décroissant
produits_volaille = produits_volaille.sort_values(by='Valeur', ascending=False)

produits_volaille.head()

### Vérification des doublons <a class="anchor" id="section_1_2_3"></a>

In [ ]:
# on recherche les doublons dans dispo_alim
duplicates = dispo_alim[dispo_alim.duplicated()]
print(duplicates)

# on filtre produits_volaille en fonction des indices des doublons dans dispo_alim
produits_volaille_filtered = produits_volaille[produits_volaille.index.isin(duplicates.index)]
print(produits_volaille_filtered)

Aucun doublon n'est à déclarer dans les deux tableaux.

### Vérification des données NaN <a class="anchor" id="section_1_2_4"></a>

In [ ]:
# vérification de la quantité de données NaN
print(produits_volaille.isnull().sum())
print ('')
print(population.isnull().sum())

On ne remarque aucune donnée manquante, ce qui est un bon signe.

## Organisation des données <a class="anchor" id="section_1_3"></a>

### Pivot <a class="anchor" id="section_1_3_1"></a>

Afin de rechercher de manière efficace dans les données, nous allons effectuer un pivot sur le dataframe dispo_alim, en ciblant la colonne "Produit". Cela nous permettra de définir les colonnes selon l'utilisation des produits alimentaires (import, export, disponibilité...)

In [ ]:
produits_volaille.head()

In [ ]:
# on pivote le df pour obtenir une seule ligne par pays et produit
produits_volaille_pivot = produits_volaille.pivot_table(index=['Zone', 'Année'],
                                                      columns='Élément',
                                                      values='Valeur',
                                                      aggfunc='sum')

# on réinitialise l'index
produits_volaille_pivot = produits_volaille_pivot.reset_index()

# on remplacer les valeurs NaN par des zéros si nécessaire
produits_volaille_pivot = produits_volaille_pivot.fillna(0)

# affichage des données
produits_volaille_pivot.head()

En utilisant l'équation suivante, on peut ajuster l'exportation, où beaucoup de données sont manquantes :

Calcul de l'exportation: Production - Disponibilité intérieure  + Importations + Variation de stock + Autres utilisations - Nourriture animale - Pertes

In [ ]:
# calcul de la colonne Exportations
produits_volaille_pivot['Exportations - Quantité'] = produits_volaille_pivot['Production'] + produits_volaille_pivot['Importations - Quantité'] - produits_volaille_pivot['Disponibilité intérieure'] + produits_volaille_pivot['Variation de stock'] + produits_volaille_pivot['Autres utilisations (non alimentaire)'] - produits_volaille_pivot['Aliments pour animaux'] - produits_volaille_pivot['Pertes']

# affichage
produits_volaille_pivot.head()

### Fusion des tableaux <a class="anchor" id="section_1_3_2"></a>

In [ ]:
# rattachement de la table population à la table produits_volaille
pop_et_dispo = pd.merge(population, produits_volaille_pivot, on=['Zone', 'Année'], how='inner')
pop_et_dispo.head()

Pour mieux comprendre et utiliser les données par la suite, nous allons renommer la colonne "Valeur" en "Population", et diviser chaque autre colonne par le nombre d'habitants. Cela nous permettra de nous passer de la population dans le prochain calcul d'outliers. Nous conservons tout de même cette dernière pour s'assurer de certaines corrélations plus tard.

In [ ]:
# la colonne 'Valeur' devient 'Population'
pop_et_dispo = pop_et_dispo.rename(columns={'Valeur': 'Population'})

# on multiplie la populaion par 1000, car la meusre était en millier d'habitants
pop_et_dispo['Population'] = pop_et_dispo['Population'] * 1000

# on liste les colonnes quantitatives (sauf 'Population')
colonnes_quantitatives = pop_et_dispo.columns[(pop_et_dispo.dtypes != 'object') & (pop_et_dispo.columns != 'Population')]

# on divise chaque colonne quantitative par la colonne 'Population'
pop_et_dispo[colonnes_quantitatives] = pop_et_dispo[colonnes_quantitatives].div(pop_et_dispo['Population'], axis=0)

# et on affiche le résultat
pop_et_dispo.head()

La modification du tableau suite au pivot fait ressortir de nouvelles colonnes inutiles dans notre analyse. En effet, certaines utilisation des produits n'auront aucun impact dans les futurs résultats, et limiter le nombre de variables permet de réduire en conséquence la perte des données après l'ACP.

In [ ]:
# on supprime les nouvelles colonnes inutiles
columns_to_drop = ['Année', 'Alimentation pour touristes', 'Aliments pour animaux', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Autres utilisations (non alimentaire)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'Pertes', 'Résidus', 'Semences', 'Nourriture', 'Traitement', 'Variation de stock']
pop_et_dispo = pop_et_dispo.drop(columns=columns_to_drop)
pop_et_dispo.head()

In [ ]:
# on conserve les données qui nous semblent le plus intéressantes
colonnes_interessantes = ['Disponibilité intérieure', 'Importations - Quantité', 'Production', 'Exportations - Quantité', 'Disponibilité alimentaire en quantité (kg/personne/an)']

# création du sous-df
fusion_simplifiée = pop_et_dispo[colonnes_interessantes]

# affichage d'un boxplot par colonnes
plt.figure(figsize=(10, 6))
fusion_simplifiée.boxplot()
plt.title("Boxplot pour les colonnes sélectionnées")
plt.xlabel("Colonnes")
plt.ylabel("Valeurs")
plt.xticks(rotation=45)  # pour faire pivoter les noms de colonnes si nécessaire
plt.show()

On distingue des outliers dans les colonnes "Importations", "Exportations" et "Disponibilité alimentaire". Nous devons donc effectuer une gestion sur ces trois colonnes principalement.

Pour rappel, la colonne "Population" n'as pas besoin d'être traitée, car nous avons uniformisé les données selon le nombre d'habitants précédemment.

### Gestion des outliers <a class="anchor" id="section_1_3_3"></a>

In [ ]:
# méthode des écarts interquartiles pour analyser les outliers des valeurs d'importation
Q1_import = pop_et_dispo['Importations - Quantité'].quantile(0.25)
Q3_import = pop_et_dispo['Importations - Quantité'].quantile(0.75)
IQR_import = Q3_import - Q1_import

lower_bound_import = Q1_import - 1.5 * IQR_import
upper_bound_import = Q3_import + 1.5 * IQR_import

outliers_import = pop_et_dispo[(pop_et_dispo['Importations - Quantité'] < lower_bound_import) | (pop_et_dispo['Importations - Quantité'] > upper_bound_import)]

# méthode des écarts interquartiles pour analyser les outliers des valeurs d'exportation
Q1_export = pop_et_dispo['Exportations - Quantité'].quantile(0.25)
Q3_export = pop_et_dispo['Exportations - Quantité'].quantile(0.75)
IQR_export = Q3_export - Q1_export

lower_bound_export = Q1_export - 1.5 * IQR_export
upper_bound_export = Q3_export + 1.5 * IQR_export

outliers_export = pop_et_dispo[(pop_et_dispo['Exportations - Quantité'] < lower_bound_export) | (pop_et_dispo['Exportations - Quantité'] > upper_bound_export)]

# méthode des écarts interquartiles pour analyser les outliers des valeurs de disponibilité alimentaire
Q1_alim = pop_et_dispo['Disponibilité alimentaire en quantité (kg/personne/an)'].quantile(0.25)
Q3_alim = pop_et_dispo['Disponibilité alimentaire en quantité (kg/personne/an)'].quantile(0.75)
IQR_alim = Q3_alim - Q1_alim

lower_bound_alim = Q1_alim - 1.5 * IQR_alim
upper_bound_alim = Q3_alim + 1.5 * IQR_alim

outliers_alim = pop_et_dispo[(pop_et_dispo['Disponibilité alimentaire en quantité (kg/personne/an)'] < lower_bound_alim) | (pop_et_dispo['Disponibilité alimentaire en quantité (kg/personne/an)'] > upper_bound_alim)]

# on concat les 3 df en un seul
outliers = pd.concat([outliers_import, outliers_export, outliers_alim])
outliers.head()

In [ ]:
# on souhaite savoir combien de pays ont été considérés comme outliers
nombre_total_outliers = len(outliers)
print(f"Le nombre total d'outliers gérés est : {nombre_total_outliers}")

In [ ]:
# suppression des outliers
data_sans_outliers = pop_et_dispo.drop(outliers.index)
data_sans_outliers.head()

In [ ]:
# création des boxplots pour vérification de l'importation
plt.figure(figsize=(10, 2))
plt.boxplot(data_sans_outliers['Importations - Quantité'], vert=False)
plt.xlabel('Importations - Quantité')
plt.title('Évaluation de l\'importation par pays')
plt.show()

# création des boxplots pour vérification de l'exportation
plt.figure(figsize=(10, 2))
plt.boxplot(data_sans_outliers['Exportations - Quantité'], vert=False)
plt.xlabel('Exportations - Quantité')
plt.title('Évaluation de l\'exportation par pays')
plt.show()

# création des boxplots pour vérification de la disponibilité alimentaire
plt.figure(figsize=(10, 2))
plt.boxplot(data_sans_outliers['Disponibilité alimentaire en quantité (kg/personne/an)'], vert=False)
plt.xlabel('Disponibilité alimentaire en quantité (kg/personne/an)')
plt.title('Évaluation de la disponibilité alimentaire par pays')
plt.show()

Le boxplot permet de confirmer que les outliers ont bien été gérés. Cela nous permettra d'utiliser plus bas un algorithme d'analyse plus performant que robuste.

# Analyse composantes - clusters <a class="anchor" id="chapter2"></a>

## Analyse des composantes <a class="anchor" id="section_2_1"></a>

### Analyse des Composantes Principales <a class="anchor" id="section_2_1_1"></a>

In [ ]:
# création d'un df copie, qui nous sera utile pour la fin
data_fin = data_sans_outliers.copy()

In [ ]:
# sélection des colonnes à standardiser
cols_to_scale = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# création d'un sous-df avec les colonnes sélectionnées
data_to_scale = data_sans_outliers[cols_to_scale]

# standardisation avec StandardScaler
scaler = StandardScaler()
data_standard = scaler.fit_transform(data_to_scale)

# création d'un df avec les données standardisées
data_standard = pd.DataFrame(data_standard, columns=cols_to_scale)

# mise à l'échelle avec MinMaxScaler pour mettre les valeurs standardisées dans la plage [-1, 1]
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
data_scaled = min_max_scaler.fit_transform(data_standard)

# création d'un df avec les données mises à l'échelle
data_scaled_df = pd.DataFrame(data_scaled, columns=cols_to_scale)

# on affiche les boxplots avant et après standardisation
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.boxplot(data_to_scale, vert=False)
plt.title('Avant Standardisation')
plt.xlabel('Colonnes')
plt.ylabel('Valeurs')

plt.subplot(1, 2, 2)
plt.boxplot(data_scaled_df, vert=False)
plt.title('Après Standardisation et Mise à l\'échelle')
plt.xlabel('Colonnes')
plt.ylabel('Valeurs')

plt.tight_layout()
plt.show()

Les données sont bien normalisées, on peut donc partir sur une ACP sans souci d'échelle.

La valeur 1 en ordonnée représente le nombre d'habitants, d'où la conservation des deux données aberrantes. Pour rappel, la population outlier n'a pas été gérée car ce n'est pas un critère d'élimination. On veut éliminer les pays où la demande sera faible, quel que soit leur nombre d'habitants.

In [ ]:
# on sélectionne les colonnes à inclure dans l'ACP
data_to_plot = data_scaled_df[['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']]

# on effectue l'ACP pour réduire la dimension à 2 composantes principales
pca = PCA(n_components=2)
data_2d = pca.fit_transform(data_to_plot)

# on crée un nuage de points 2D
plt.figure(figsize=(8, 6))
plt.scatter(data_2d[:, 0], data_2d[:, 1])
plt.title('Nuage de points après ACP (2D)')
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')

# on affiche les noms de pays ou d'observations
for i, txt in enumerate(data_sans_outliers['Zone']):
    plt.annotate(txt, (data_2d[i, 0], data_2d[i, 1]))

plt.show()

In [ ]:
# sélection des colonnes à inclure dans l'ACP (sans la population)
data_to_plot = data_scaled_df[['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']]

# analyse de l'ACP réduite à 3 dimensions
pca = PCA(n_components=3)
data_3d = pca.fit_transform(data_to_plot)

# création du graph 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

x = data_3d[:, 0]
y = data_3d[:, 1]
z = data_3d[:, 2]

ax.scatter(x, y, z)
ax.set_xlabel('Composante Principale 1')
ax.set_ylabel('Composante Principale 2')
ax.set_zlabel('Composante Principale 3')
ax.set_title('Nuage de points après ACP (3D)')

# on affiche les noms des pays
for i, txt in enumerate(data_sans_outliers['Zone']):
    ax.text(x[i], y[i], z[i], txt)

plt.show()

Cette analyse n'apporte rien en terme de chiffres, mais une visualisation en 3 dimensions permet de distinguer des écarts entre pays que l'absence de profondeur ne permet pas de repérer.

In [ ]:
# on crée la matrice de corrélation
matrice_cor = data_scaled_df[cols_to_scale].corr()

# création de la heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(matrice_cor, annot=True, cmap="coolwarm", square=True, fmt=".2f", linewidths=.5)
plt.title("Matrice de Corrélation")
plt.show()

Cette méthode permet de repérer les corrélations entre les différentes dimensions. Cependant, elle n'est pas la plus lisible pour comparer toutes les dimensions en même temps. Essayons une autre méthode, mais gardons en tête certains liens.

### Biplot et charges factorielles <a class="anchor" id="section_2_1_2"></a>

In [ ]:
# on sélectionne les colonnes à inclure dans l'ACP
cols_to_scale = ['Population', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# standardisation des colonnes sélectionnées
scaler = StandardScaler()
data_sans_outliers[cols_to_scale] = scaler.fit_transform(data_sans_outliers[cols_to_scale])

# réalisation de l'ACP
pca = PCA()
data_pca = pca.fit_transform(data_sans_outliers[cols_to_scale])

# on cherche les valeurs de variance_ratio
variance_ratios = pca.explained_variance_ratio_

# calcul du nombre de composantes nécessaires pour atteindre 90% de variance
cumulative_variance = np.cumsum(variance_ratios)
n_components_for_90_percent_variance = np.argmax(cumulative_variance >= 0.90) + 1

# graphique du cercle des charges factorielles
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.axhline(0, color='gray', linewidth=0.5)
ax.axvline(0, color='gray', linewidth=0.5)

# tracé un cercle
circle = plt.Circle((0, 0), 1, fill=False, color='gray')
ax.add_patch(circle)

# tracé des flèches pour chaque variable
for i, col in enumerate(cols_to_scale):
    x = pca.components_[0, i]
    y = pca.components_[1, i]
    ax.arrow(0, 0, x, y, head_width=0.05, head_length=0.05, fc='r', ec='r')
    ax.text(x, y, col, fontsize=12, ha='left', va='bottom')

# titres et labels
ax.set_title('Cercle des Charges Factorielles')
ax.set_xlabel('Composante Principale 1')
ax.set_ylabel('Composante Principale 2')

# Affichage du pourcentage de variance expliquée
percentage_variance = np.sum(variance_ratios[:2]) * 100
ax.text(1.05, 0, f'Variance expliquée :\n{percentage_variance:.2f}%', fontsize=12, ha='left', va='center', color='b')

plt.grid()
plt.show()
print()
print(f"Pour atteindre 90% de variance expliquée, il faut conserver {n_components_for_90_percent_variance} composantes principales.")


Ce graphique est beaucoup plus parlant et montre plusieurs choses :

- La population est inversement proportionnelle à l'importation et la disponibilité alimentaire, mais une corrélation positive légère se fait avec la production et la disponibilité intérieure
- L'exportation a une corrélation faible avec les autres composantes
- La production et la disponibilité intérieure sont fortement correlées et très bien représentées
- L'importation et la dispo par personne sont aussi fortement correlées et très bien représentées
- Il ne semble pas y avoir de lien entre l'axe "production/dispo intérieure" et l'axe "importations/dispo par pers.".

Afin d'avoir un point de vue encore meilleur, pourquoi ne pas superposer le graphique des charges factorielles et celui de l'ACP?

In [ ]:
# sélection des colonnes à inclure dans l'ACP
cols_to_scale = ['Population', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# standardisation des colonnes sélectionnées
scaler = StandardScaler()
data_sans_outliers[cols_to_scale] = scaler.fit_transform(data_sans_outliers[cols_to_scale])

# réalisation de l'ACP
pca = PCA()
pca.fit(data_sans_outliers[cols_to_scale])

# obtention des charges factorielles
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# idem pour les coordonnées des pays dans l'espace des composantes principales
pcs = pca.transform(data_sans_outliers[cols_to_scale])

# on crée un biplot
fig, ax = plt.subplots(figsize=(10, 10))

# on trace les flèches des charges factorielles
for i, col in enumerate(cols_to_scale):
    x = loadings[i, 0]
    y = loadings[i, 1]
    ax.arrow(0, 0, x, y, head_width=0.05, head_length=0.05, fc='r', ec='r')
    ax.text(x, y, col, fontsize=12, ha='left', va='bottom')

# on place les pays dans l'espace ACP
ax.scatter(pcs[:, 0], pcs[:, 1], alpha=0.5, label='Observations')

# titres et labels
ax.set_title('Biplot des Composantes Principales')
ax.set_xlabel('Composante Principale 1')
ax.set_ylabel('Composante Principale 2')
ax.legend()

plt.grid()
plt.show()

La direction des flèches indique la direction et la force des variables originales dans l'espace des composantes principales. La répartition des pays dans ce "biplot" indique leur positionnement par rapport aux variables.

Ainsi :

   La flèche de la population pointant vers le haut signifie que les pays avec des valeurs élevées dans cette variable sont situés plus haut sur l'axe des composantes principales 2.
    Les flèches d'importation et de disponibilité alimentaire pointant vers le bas à droite signifient que les pays avec des valeurs élevées dans ces variables sont situés dans le coin bas-droite de l'espace des composantes principales.

Ces identifications nous permettent de distinguer d'emblée des groupes de pays, mais aussi de "prévoir" les pays qui peuvent nous intéresser. Dans notre cas, nous allons nous concentrer notamment sur les pays qui importent beaucoup, où la demande est la plus importante.

Cas particulier:
Le cluster au centre-gauche, où aucune flèche ne pointe directement, peut indiquer que ces observations partagent des caractéristiques spécifiques qui ne sont pas fortement influencées par les variables incluses dans l'ACP, ou encore que les variables qui les définissent le plus ne sont pas les mieux représentées.

Afin de continuer notre analyse, il nous faut définir le nombre de composantes à conserver, le tout sans perdre trop de données. Le cercle des charges factorielles était accompagné d'une analyse de la variance cumulative qui estime que 4 composantes suffisent à conserver 90% de nos informations. Essayons une autre méthode pour voir si le résultat diffère.

### Éboulis des valeurs propres et coude  <a class="anchor" id="section_2_1_3"></a>

In [ ]:
# récupération des valeurs propres à partir de l'ACP
eigenvalues = pca.explained_variance_

# calcul de la variance cumulée normalisée
cumulative_variance = np.cumsum(eigenvalues) / np.sum(eigenvalues)

# on cherche le coude
elbow_index = np.argmax(np.diff(cumulative_variance) < 0.01) + 1

# création du graph d'éboulis des valeurs propres avec variance cumulée et coude
fig, ax1 = plt.subplots(figsize=(8, 6))

# barres pour les valeurs propres
ax1.bar(range(1, len(eigenvalues) + 1), eigenvalues, align='center', label='Valeur Propre', alpha=0.7)
ax1.set_xlabel('Composante Principale')
ax1.set_ylabel('Valeur Propre', color='tab:blue')
ax1.tick_params('y', colors='tab:blue')

# ligne pour la variance cumulée
ax2 = ax1.twinx()
ax2.plot(range(1, len(eigenvalues) + 1), cumulative_variance, label='Variance Cumulée', marker='o', color='r')
ax2.set_ylabel('Variance Cumulée', color='tab:red')
ax2.tick_params('y', colors='tab:red')

# point pour le coude
ax2.scatter(elbow_index, cumulative_variance[elbow_index - 1], c='green', marker='o', s=100, label='Coude')

plt.title('Graphique d\'Éboulis des Valeurs Propres avec Variance Cumulée et Coude')
plt.legend()
plt.show()

D'emblée, deux informations ressortent :
- Le coude est à la 5eme composante
- la première composante est largement plus impactante que les autres

Les deux analyses précédentes ont révélé toutefois un résultat différent :
- La méthode du coude indique que 5 composantes seraient nécessaires à l'ACP
- La méthode des variances cumulatives estime que 4 composantes sont suffisantes pour conserver plus de 90% de nos données.

Afin de limiter au maximum la perte des données, nous allons conserver 5 composantes. Par ailleurs, en rapport avec ce qui a été dit plus tôt, nous ne prendrons pas en compte la poulation dans la gestion des clusters. Il nous reste alors 5 composantes :
- La production
- L'importation
- L'exportation
- La disponibilité intérieure
- La disponibilité par personne

Étape suivante : la définition du nombre de clusters.

## Analyse des clusters <a class="anchor" id="section_2_2"></a>

### Analyse via des modèles arbitraires <a class="anchor" id="section_2_2_1"></a>

In [ ]:
# on sélectionne les colonnes à inclure dans l'analyse K-Means
X = data_sans_outliers[cols_to_scale]

# test de différentes valeurs de k et enregistrement du résultat
k_values = range(2, 11)
inertias = []
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))

# on trace les graphique du coude et de la silhouette
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(k_values, inertias, 'o-')
plt.title('Méthode du Coude')
plt.xlabel('Nombre de Clusters (k)')
plt.ylabel('Inertie')

plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, 'o-')
plt.title('Score de Silhouette')
plt.xlabel('Nombre de Clusters (k)')
plt.ylabel('Score de Silhouette')

plt.tight_layout()
plt.show()

La méthode du coude ne montre pas de cassure nette, ce qui rend le choix du nombre optimal de clusters difficile.

Le score de Silhouette quant à lui est censé indiquer l'organisation des clusters, car il mesure à quel envergure chaque point d'un cluster est similaire aux autres points du même cluster, notamment par rapport aux points des clusters voisins. Le score varie de -1 à 1, où une valeur élevée indique que l'objet est bien adapté à son propre cluster et mal adapté aux clusters voisins.

    Un score proche de 1 indique une bonne configuration du clustering.
    Un score proche de 0 indique un chevauchement de clusters.
    Un score proche de -1 indique une mauvaise affectation au cluster.

Sur le papier, c'est bien joli, mais notre graphique ne permet pas une lecture optimale de cette organisation: 

La courbe monte et descend beaucoup, ce qui peut indiquer une certaine ambiguïté dans la structure des données. Par ailleurs, l'oscillation du score entre 0.41 et 0.34, avec de fortes cassures, indique un flou dans la structure des clusters.
La justification peut provenir de la forme irrégulière des clusters, des données d'origine... Quoiqu'il en soit, cela rend le choix du nombre optimal de clusters délicat.

Nous allons tenter une autre méthode "automatique" de sélection des clusters : l'analyse des inerties.

In [ ]:
# reprise de l'ACP
pca = PCA()
principal_components = pca.fit_transform(data_sans_outliers[cols_to_scale])

# mise en place d'un modèle KMeans avec la méthode partielle
kmeans_partial = MiniBatchKMeans(n_clusters=k)

# calcul des inerties pour différentes valeurs de k
inertia_values = []
cluster_range = range(1, 11)  # choix d'une plage appropriée de clusters
for k in cluster_range:
    kmeans_partial.partial_fit(principal_components)
    inertia_values.append(kmeans_partial.inertia_)

# tracé de la courbe d'inerties
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, inertia_values, marker='o')
plt.title('Analyse des Inerties')
plt.xlabel('Nombre de Clusters')
plt.ylabel('Inertie')
plt.grid(True)
plt.show()

Encore une fois, le résultat est ambigu. Trois, quatre ou cinq clusters? Rien ne permet de choisir le bon nombre. Essayons autre chose : le DBSCAN

In [ ]:
# sélection des colonnes à inclure dans l'ACP
cols_to_scale = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# standardisation des colonnes sélectionnées
scaler = StandardScaler()
data_sans_outliers[cols_to_scale] = scaler.fit_transform(data_sans_outliers[cols_to_scale])

# on réalise l'ACP
pca = PCA()
pca.fit(data_sans_outliers[cols_to_scale])

# on cherche les composantes principales
principal_components = pca.transform(data_sans_outliers[cols_to_scale])

# utilisation de DBSCAN sur les composantes principales
dbscan = DBSCAN(eps=1, min_samples=5)
clusters_dbscan = dbscan.fit_predict(principal_components)

# on crée un nuage de points coloré par cluster
plt.scatter(principal_components[:, 0], principal_components[:, 1], c=clusters_dbscan, cmap='viridis')
plt.title('Résultat de DBSCAN sans la colonne Population')
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.show()

La méthodes du DBSCAN nous révèle 3 groupes très dispersés, dont le plus important (le violet) s'éparpille dans la moitié droite du cercle des charges factorielles, et se voit concerné par toutes les composantes principales.

Il nous faudrait utiliser un autre algorithme, quitte à choisir volontairement le nombre de clusters, afin d'avoir des cercles plus fermés et ainsi limiter le nombre d'individus. Outre une sélection plus précise, nous pourrons peut être voir une corrélation entre chaque cluster et l'ACP.

### Dendrogramme, ou Classification Ascendante Hiérarchique <a class="anchor" id="section_2_2_2"></a>

In [ ]:
# on exclut la variable 'Population'
data_sans_population = data_sans_outliers.drop('Population', axis=1)

# on garde uniquement les colonnes numériques
data_sans_population_numeric = data_sans_population.select_dtypes(include='number')

# calcul de la matrice de liaison
linkage_matrix_sans_population = linkage(data_sans_population_numeric, method='ward')

# définition d'un "seuil de coupe"
t = 14

# création d'un dendrogramme horizontal
fig, ax = plt.subplots(figsize=(12, 20))
dendrogram(linkage_matrix_sans_population, labels=data_sans_population['Zone'].tolist(), orientation='left', leaf_rotation=0, leaf_font_size=8, color_threshold=0.8*t)
plt.title('Dendrogramme des Pays basé sur l\'ACP')
plt.xlabel('Distance Euclidienne')
plt.ylabel('Pays')
plt.show()

La sélection de t est arbitraire, mais permet de comparer les différents pays en les séparant en 4 clusters. Par ailleurs, un certain nombre des pays inclus dans le cluster rouge correspond aux pays ayant la plus forte valeur d'importation, comme vu dans un des scatterplots plus haut.

Testons avec la méthode du kmeans, afin de pouvoir comparer nos résultats à une autre source et, éventuellement, s'assurer de la stabilité de nos clusters.

### Algorithme Kmeans <a class="anchor" id="section_2_2_3"></a>

In [ ]:
# sélection des colonnes à inclure dans l'ACP
cols_to_scale = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# standardisation des les colonnes sélectionnées
scaler = StandardScaler()
data_sans_outliers[cols_to_scale] = scaler.fit_transform(data_sans_outliers[cols_to_scale])

# on réalise l'ACP
pca = PCA()
principal_components = pca.fit_transform(data_sans_outliers[cols_to_scale])

# choix défini du nombre de clusters
k = 4

# on crée l'objet KMeans avec n_init élevé et random_state fixé
kmeans = KMeans(n_clusters=k, n_init=50, random_state=42)

# ajustement du modèle
kmeans.fit(principal_components)

# on récupère les étiquettes de cluster
cluster_labels = kmeans.labels_

# coordonnées des centroïdes
centroids = kmeans.cluster_centers_

# création d'un nuage de points coloré par cluster
plt.figure(figsize=(14, 12))
plt.scatter(principal_components[:, 0], principal_components[:, 1], c=cluster_labels, cmap='viridis')
plt.scatter(centroids[:, 0], centroids[:, 1], marker='X', s=200, c='red')  # centroides en rouge
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.title('Clustering des Pays basé sur l\'ACP avec Centroides')
plt.grid(True)

# noms des pays
for i, txt in enumerate(data_sans_outliers['Zone']):
    plt.annotate(txt, (principal_components[i, 0], principal_components[i, 1]))

# affichage
plt.show()

Nos clusters sont bien représentés, et le centroïde du cluster bleu va clairement dans la direction de l'importation, ce qui semble prometteur.

### Analyse des résultats <a class="anchor" id="section_2_2_4"></a>

Analysons les différents résultats obtenus :

In [ ]:
# on appliquer la découpe de l'arbre en fonction du seuil (t) précédent
clusters_dendrogramme = cut_tree(linkage_matrix_sans_population, height=10).flatten()

# ajout de la colonne d'étiquettes de cluster au df
data_sans_population['Cluster_dendro'] = clusters_dendrogramme

# affichage
clusters_dendro = data_sans_population.groupby(['Cluster_dendro']).mean().round(2)
clusters_dendro

In [ ]:
# ajout de la colonne d'étiquettes de cluster au df
data_sans_population['Cluster'] = cluster_labels

clusters_kmeans = data_sans_population.groupby(['Cluster']).mean().round(2)
clusters_kmeans = clusters_kmeans.drop('Cluster_dendro', axis=1)
clusters_kmeans

In [ ]:
# suppression des lignes où les résultats des méthodes de clustering sont les mêmes
clusters_différents = data_sans_population[data_sans_population['Cluster'] != data_sans_population['Cluster_dendro']]

# affichage du nombre de pays ayant "changé" de clusters
nombre_de_lignes = clusters_différents.shape[0]
print(f"Nombre de lignes : {nombre_de_lignes}")

On distingue dans les tableaux d'analyse de moyenne une inversion entre le cluster 1 et le cluster 3, sans compter quelques petites différences entre les chiffres des clusters "fixes". Ceci permet de comprendre pourquoi 59 pays semblent avoir changé de cluster.

Les écarts de chiffres sont justifiés par la nature même du kmeans et du dendrogramme. Si le premier est basée sur la distance euclidienne entre les différents points, ce n'est pas le cas du second, qui fonctionne sur les relations de similarités entre les composantes, notamment grâce à la "distance de Ward" que nous avons utilisée.

Analysons plus en profondeur les caractéristiques des clusters :

In [ ]:
# sélection des colonnes à inclure dans la heatmap
cols_to_compare = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# création d'un df avec les colonnes sélectionnées
heatmap_data = data_sans_population[cols_to_compare + ['Cluster']]

# on cré la heatmap en inversant les axes et en inclinant les noms
plt.figure(figsize=(12, 6))
heatmap = sns.heatmap(heatmap_data.groupby(['Cluster']).mean(), annot=True, cmap='coolwarm', xticklabels=cols_to_compare, yticklabels='auto')
plt.title('Heatmap des Colonnes avec les Clusters')

# on tilt les abscisses de 45°
plt.xticks(rotation=45, ha='right')

# on horizontalise les ordonnées (pas de base?)
plt.yticks(rotation=0, ha='right')

plt.show()

La clustermap indique clairement les corrélations avec les composantes de notre dataframe, ce qui nous permet de confirmer que le cluster 2 est le plus intéressant car c'est le regroupement de pays le plus demandeur de ressources. Au contraire, les valeurs des autres clusters sont négatives, ce qui indique que ces clusters sont majoritairement excédents.

In [ ]:
# Sélection des colonnes à inclure dans les boxplots
cols_to_compare = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# Création d'un DataFrame avec les colonnes sélectionnées et le cluster
boxplot_data = data_sans_population[cols_to_compare + ['Cluster']]

# Configuration de la taille de la figure
plt.figure(figsize=(14, 8))

# Boucle sur chaque colonne pour créer des boxplots par cluster
for i, col in enumerate(cols_to_compare):
    plt.subplot(2, 3, i + 1)  # Réglage de la disposition des sous-plots
    sns.boxplot(x='Cluster', y=col, data=boxplot_data, palette='viridis')
    plt.title(f'{col}')

# Ajustement du placement des sous-plots
plt.tight_layout()

# Affichage de la figure
plt.show()


L'aperçu par boxplot est un autre moyen visuel de comparer les différents clusters par composante et conforte à nouveau les résultats précédemment obtenus. Maintenant que nous avons déterminé le regroupement de pays le plus intéressant pour notre étude de marché, recherchons le pays le plus "attractif".

### Évaluation du cluster conservé <a class="anchor" id="section_2_2_5"></a>

In [ ]:
# Trier les données selon le cluster
sorted_data = data_sans_population.sort_values(by='Cluster')

# Filtrer uniquement les données du cluster 2
données_cluster2 = sorted_data[sorted_data['Cluster'] == 2]

# Afficher les données du cluster 2
données_cluster2

Les valeurs sont encore celles qui ont été standardisées précedemment. Il nous faudra les remplacer par les données d'origine. Pour ce faire, nous allons supprimer les colonnes modifiées de données_cluster2, et le fusionner avec data_fin, que nous avions mis de coté avant de faire la normalisation.

Accessoirement, nous pouvons voir que tous les pays, excepté la Namibie et la Géorgie, ont bien été gérés de la même manière par le dendrogramme et kmeans.

In [ ]:
# suppression des colonnes standardisées
colonnes_à_supprimer = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production', 'Cluster_dendro']

données_cluster2 = données_cluster2.drop(columns=colonnes_à_supprimer)
données_cluster2

In [ ]:
# fusion des df cluster2 clean et de data_fin, qu'on avait conservé
data_origin_c2 = pd.merge(données_cluster2, data_fin, on=['Zone'], how='inner')

# liste des colonnes à modifier suite à la division par habitants
colonnes_a_inverser = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Production']

# boucle sur chaque colonne à inverser
for colonne in colonnes_a_inverser:
    # multiplication par la colonne 'Population'
    data_origin_c2[colonne] = data_origin_c2[colonne] * data_origin_c2['Population']
    
# organisation par importation décroissante
data_origin_c2 = data_origin_c2.sort_values(by='Importations - Quantité', ascending=False)

data_origin_c2

In [ ]:
# sauvegarde du fichier final
data_origin_c2.to_csv(r"C:\Users\Avenmythril\Desktop\Formations\DAn\Projet 9\Talbot_Robin_112023_P9.csv")

In [ ]:
import plotly.express as px

# Créer un dictionnaire de correspondance entre les noms de pays français et ISO A3
correspondance_iso = {
    'Allemagne': 'DEU',
    'Iraq': 'IRQ',
    'Arabie saoudite': 'SAU',
    'Cuba': 'CUB',
    'Tchéquie': 'CZE',
    'Autriche': 'AUT',
    'Bulgarie': 'BGR',
    'Irlande': 'IRL',
    'Congo': 'COG',
    'Suède': 'SWE',
    'Slovaquie': 'SVK',
    'Géorgie': 'GEO',
    'Macédoine du Nord': 'MKD',
    'Albanie': 'ALB',
    'Jamaïque': 'JAM',
    'Arménie': 'ARM',
    'Croatie': 'HRV',
    'Namibie': 'NAM',
    'Slovénie': 'SVN',
    'Timor-Leste': 'TLS',
    'Îles Salomon': 'SLB'
}

# Appliquer la correspondance aux noms de pays dans data_origin_c2
data_origin_c2['ISO_A3'] = data_origin_c2['Zone'].map(correspondance_iso)

# Utiliser la colonne 'ISO_A3' pour les codes ISO A3 avec une plage de couleurs ajustée
fig = px.choropleth(
    data_origin_c2,
    locations='ISO_A3',
    color='Importations - Quantité',
    hover_name='Zone',
    title='Carte des pays à forte importation',
    color_continuous_scale='Reds',
    projection='natural earth',
    range_color=[data_origin_c2['Importations - Quantité'].min(), data_origin_c2['Importations - Quantité'].max()]  # Ajuster la plage de couleurs
)

fig.show()


In [ ]:
# Créer un dictionnaire de correspondance entre les noms de pays français et anglais
correspondance_pays = {
    'Allemagne': 'Germany',
    'Iraq': 'Iraq',
    'Arabie saoudite': 'Saudi Arabia',
    'Cuba': 'Cuba',
    'Tchéquie': 'Czechia',
    'Autriche': 'Austria',
    'Bulgarie': 'Bulgaria',
    'Irlande': 'Ireland',
    'Congo': 'Congo',
    'Suède': 'Sweden',
    'Slovaquie': 'Slovakia',
    'Géorgie': 'Georgia',
    'Macédoine du Nord': 'North Macedonia',
    'Albanie': 'Albania',
    'Jamaïque': 'Jamaica',
    'Arménie': 'Armenia',
    'Croatie': 'Croatia',
    'Namibie': 'Namibia',
    'Slovénie': 'Slovenia',
    'Timor-Leste': 'Timor-Leste',
    'Îles Salomon': 'Solomon Islands'
}

# Appliquer la correspondance aux noms de pays dans data_origin_c2
data_origin_c2['Zone_EnAnglais'] = data_origin_c2['Zone'].map(correspondance_pays)

# Charger le fichier de formes des pays
path_to_shapefile = r"C:\Users\Avenmythril\Desktop\Formations\DAn\Projet 9\110m_cultural\ne_110m_admin_0_countries.shp"
world = gpd.read_file(path_to_shapefile)

# Fusionner les données du tableau avec les données géospatiales
merged = world.merge(data_origin_c2, how='left', left_on='NAME', right_on='Zone_EnAnglais', indicator=True)

# Créer la colonne 'InDataFrame' pour marquer les pays dans votre DataFrame
merged['InDataFrame'] = merged['_merge'] == 'both'

# Plot avec GeoPandas et Matplotlib
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
merged.plot(column='InDataFrame', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
plt.title('Carte des pays à forte importation', fontsize=16)
plt.show()
